# Image Classification with Fine-Tuned VGG16
This notebook demonstrates an end-to-end pipeline for image classification using transfer learning with the VGG16 model.

## Import Libraries
The first step is to import the necessary libraries. These include libraries for:
- Data processing (`numpy`, `pandas`)
- Visualization (`matplotlib`, `seaborn`)
- Image handling (`skimage`)
- Deep learning (`tensorflow`, `keras`)
- Progress tracking (`tqdm`)

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras.optimizers import Adam

from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K


## Data Augmentation and Dataset Preparation
Data augmentation is used to increase the variability of the training data by applying random transformations like rotation, flipping, etc. Here, we also prepare datasets for training, validation, and testing.

In [ ]:

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Deep Learning /Data/train',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64
)

valid_dataset = valid_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Deep Learning /Data/valid',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64
)

test_dataset = test_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Deep Learning /Data/test',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64
)


## Model Creation
This section builds the classification model using transfer learning. The VGG16 model is loaded with pre-trained ImageNet weights. We then add custom layers to fine-tune it for our dataset.

In [ ]:

base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Freezing layers except the last 8
for layer in base_model.layers[:-8]:
    layer.trainable = False

model = Sequential([
    base_model,
    Dropout(0.5),
    Flatten(),
    BatchNormalization(),
    Dense(32, kernel_initializer='he_uniform', activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, kernel_initializer='he_uniform', activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, kernel_initializer='he_uniform', activation='relu'),
    BatchNormalization(),
    Dense(4, activation='softmax')
])

model.summary()


## Compile and Train
The model is compiled with the Adam optimizer and categorical cross-entropy loss. Training metrics include accuracy, precision, recall, AUC, and F1-score. We use callbacks for efficient training.

In [ ]:

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    f1_score
]

lrd = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=1e-7)
mcp = ModelCheckpoint('model.h5')
es = EarlyStopping(verbose=1, patience=3)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=METRICS)

history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=20,
    verbose=1,
    callbacks=[lrd, mcp, es]
)


## Model Evaluation
Evaluate the trained model on the test dataset and display the metrics.

In [ ]:

model.evaluate(test_dataset, verbose=1)


## Visualize Metrics
Plot the training and validation metrics over epochs to analyze the model's performance.

In [ ]:

def Train_Val_Plot(acc, val_acc, loss, val_loss, auc, val_auc, precision, val_precision, f1, val_f1):
    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(20, 5))
    fig.suptitle("MODEL'S METRICS VISUALIZATION")

    ax1.plot(range(1, len(acc) + 1), acc, label='Training Accuracy')
    ax1.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
    ax1.set_title('Accuracy')
    ax1.legend()

    ax2.plot(range(1, len(loss) + 1), loss, label='Training Loss')
    ax2.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
    ax2.set_title('Loss')
    ax2.legend()

    ax3.plot(range(1, len(auc) + 1), auc, label='Training AUC')
    ax3.plot(range(1, len(val_auc) + 1), val_auc, label='Validation AUC')
    ax3.set_title('AUC')
    ax3.legend()

    ax4.plot(range(1, len(precision) + 1), precision, label='Training Precision')
    ax4.plot(range(1, len(val_precision) + 1), val_precision, label='Validation Precision')
    ax4.set_title('Precision')
    ax4.legend()

    ax5.plot(range(1, len(f1) + 1), f1, label='Training F1-Score')
    ax5.plot(range(1, len(val_f1) + 1), val_f1, label='Validation F1-Score')
    ax5.set_title('F1-Score')
    ax5.legend()

    plt.show()

Train_Val_Plot(
    history.history['accuracy'], history.history['val_accuracy'],
    history.history['loss'], history.history['val_loss'],
    history.history['auc'], history.history['val_auc'],
    history.history['precision'], history.history['val_precision'],
    history.history['f1_score'], history.history['val_f1_score']
)
